In [1]:
import json

from datasets import load_dataset

data = load_dataset("json", data_files="openvivqa_submission.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
data[0]

{'id': '940',
 'image': 'dev-images/000000000000.jpg',
 'question': '<image>\nngười dân không được tụ tập quá bao nhiêu người ?',
 'answer': '10 người',
 'predicted': 'Người dân không được tụ tập quá 10 người.'}

In [3]:
duplicate_ids = set()
for d in data:
  if d["id"] in duplicate_ids:
    print(d["id"])
  duplicate_ids.add(d["id"])

In [4]:
len(data)

3545

In [5]:
import os
from pathlib import Path

from langchain.globals import set_llm_cache
from langchain.prompts import ChatPromptTemplate
from langchain_community.cache import SQLiteCache
from langchain_core.output_parsers import StrOutputParser

from vividbot.data.task.vivid_instruct_65k.utils.llms import LLM

os.makedirs(f"{Path.home()}/.cache", exist_ok=True)
set_llm_cache(SQLiteCache(database_path=f"{Path.home()}/.cache/.langchain.db"))


def get_generate_qa_pairs_chain():
  return (
    ChatPromptTemplate.from_messages(
      [
        (
          "system",
          "Bạn là trợ lý đánh giá về độ chính xác của câu trả lời của một hệ thống AI."
          "Bạn được cung cấp một câu hỏi, một câu trả lời đúng và một câu trả lời dự đoán."
          "Nhiệm vụ của bạn là đánh giá xem câu trả lời dự đoán có đúng, gần đúng, hoặc tương tự hay không."
          "Câu trả lời đúng là câu trả lời mang thông tin đúng với trọng tâm của câu hỏi dù có thể mang nhiều thông tin khác."
          "Nếu câu trả lời dự đoán chứa thông tin đúng hoặc gần giống với ý chính của đáp án đúng và trọng tâm của câu hỏi hãy trả lời 1."
          "Các trường hợp có thể được coi là đúng bao gồm: câu trả lời chứa một phần thông tin chính xác, mô tả tương tự hoặc gần đúng với tình huống được mô tả trong đáp án đúng."
          "Nếu câu trả lời dự đoán hoàn toàn sai, không liên quan, hoặc mâu thuẫn với đáp án đúng, hãy trả lời 0."
          "Chỉ cần trả lời 0 hoặc 1 mà không cần giải thích."
          "Ví dụ: Câu hỏi: Ai là người đầu tiên đặt chân lên mặt trăng?"
          "Đáp án đúng: Neil Armstrong"
          "Câu trả lời: Neil Armstrong"
          "Trả lời: 1"
          "Ví dụ 2: Câu hỏi: Ai là người đầu tiên đặt chân lên mặt trăng?"
          "Đáp án đúng: Neil Armstrong"
          "Câu trả lời: Buzz Aldrin"
          "Trả lời: 0"
          "Ví dụ 3: Câu hỏi: Người đàn ông đang đeo gì sau lưng?"
          "Đáp án đúng: một cái ba lô"
          "Câu trả lời: Người đàn ông đang đeo một cái ba lô màu đen."
          "Trả lời: 1"
          "Ví dụ 4: Câu hỏi: Người phụ nữ mặc áo đỏ đang làm gì?"
          "Đáp án đúng: đang nói về khu bảo tàng mà các em học sinh đang tham quan"
          "Câu trả lời: Người phụ nữ mặc áo đỏ đang đứng trước một nhóm học sinh."
          "Trả lời: 1"
          "Ví dụ 5: Câu hỏi: Người đàn ông mặc áo đen đeo khẩu trang xanh da trời đang đẩy thứ gì?"
          "Đáp án đúng: người đàn ông mặc áo đen đeo khẩu trang xanh da trời đang đẩy chiếc xe đẩy siêu thị màu đỏ"
          "Câu trả lời: Người đàn ông đang đẩy một giỏ hàng."
          "Trả lời: 1",
        ),
        (
          "human",
          "Câu hỏi: {question}\n\nĐáp án đúng: {answer}\n\nCâu trả lời: {predicted}\n\nTrả lời:",
        ),
      ]
    )
    | LLM
    | StrOutputParser()
  )


def process_batch(batch):
  batch_ids = batch["id"]
  batch_images = batch["image"]
  batch_questions = batch["question"]
  batch_answers = batch["answer"]
  batch_predicteds = batch["predicted"]
  processed_data = (
    [json.loads(line) for line in open("openvivqa_scores.jsonl")]
    if os.path.exists("openvivqa_scores.jsonl")
    else []
  )
  processed_ids = [d["id"] for d in processed_data]
  for id, image, q, a, p in zip(
    batch_ids, batch_images, batch_questions, batch_answers, batch_predicteds
  ):
    if id in processed_ids and id not in duplicate_ids:
      continue
    chain = get_generate_qa_pairs_chain()
    score = float(
      chain.invoke(
        {
          "question": q.replace("<image>", "").strip(),
          "answer": a,
          "predicted": p,
        }
      )
    )
    d = {
      "id": id,
      "image": image,
      "question": q,
      "answer": a,
      "predicted": p,
      "score": score,
    }
    with open("openvivqa_scores.jsonl", "a") as f:
      f.write(json.dumps(d, ensure_ascii=False) + "\n")


data.map(process_batch, batch_size=6, batched=True, num_proc=4)

/home/dminhvu/miniconda3/envs/vividbot/lib/python3.10/site-packages/langchain_core/utils/utils.py:225: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(
/home/dminhvu/miniconda3/envs/vividbot/lib/python3.10/site-packages/langchain_core/utils/utils.py:225: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(
/home/dminhvu/miniconda3/envs/vividbot/lib/python3.10/site-packages/langchain_groq/chat_models.py:355: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(
Parameter 'function'=<function process_batch at 0x777591d3dc60> of the transform datasets.arrow_dataset.Dataset._

Map (num_proc=4):   0%|          | 0/3545 [00:00<?, ? examples/s]

I0000 00:00:1724567889.732662 1897806 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


Dataset({
    features: ['id', 'image', 'question', 'answer', 'predicted'],
    num_rows: 3545
})

In [6]:
scores = [json.loads(line) for line in open("openvivqa_scores.jsonl")]

In [7]:
correct = sum(s["score"] for s in scores)
total = len(scores)
accuracy = correct / total
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.3377


In [9]:
corrects = [s for s in scores if s["score"] == 1]

In [10]:
len(corrects)

1256

In [18]:
corrects[100]

{'id': '28234',
 'question': '<image>\nngười phụ nữ mặc áo màu vàng mang dù màu gì ?',
 'answer': 'dù màu xanh dương',
 'predicted': 'Người phụ nữ mặc áo màu vàng mang dù màu xanh dương.',
 'score': 1.0}

In [3]:
import json

d = [json.loads(line) for line in open("83k_openvivqa_scores.jsonl").readlines()]
d = sorted(d, key=lambda x: int(x["id"]))
with open("83k_openvivqa_scores_sorted.jsonl", "w") as f:
  for x in d:
    f.write(json.dumps(x, ensure_ascii=False) + "\n")